# Trades Notebook

## Overview

This program connects to a local Postgres database running on a Docker container, and puts trading data that has been loaded into the Postgres table `public.trades` into a DataFrame for analysis. The program uses the trading data in the DataFrame and calculates the number of day trades for each account over a specified range of time.

## Setup

Setups up the connection to Postgres and loads the `public.trades` data into a DataFrame.

In [7]:
import psycopg2
import pandas as pd

# create a connection to the Postgres database
connection = psycopg2.connect("dbname=postgres host=sp-psql user=postgres password=postgres")
# create a cursor
cursor = connection.cursor()

# execute a query
cursor.execute("SELECT * FROM public.trades")

# retrieve query results
records = cursor.fetchall()
cursor.close()

# define the column names
column_names = ['account_id','symbol','side','qty','timestamp']

# create a DataFrame
trades_df = pd.DataFrame(records, columns=column_names)

       account_id symbol  side  qty           timestamp
0           77658   GOOG  sell  493 2023-09-04 09:30:00
1           19191   NVDA  sell  216 2023-09-04 09:30:03
2           90209   NVDA  sell  108 2023-09-04 09:30:07
3           59797     FB   buy  612 2023-09-04 09:30:07
4           20249   NFLX   buy    7 2023-09-04 09:30:12
...           ...    ...   ...  ...                 ...
37647       38363   NFLX  sell  732 2023-09-08 15:59:35
37648       19170   MSFT   buy  726 2023-09-08 15:59:37
37649       18089   GOOG  sell  363 2023-09-08 15:59:51
37650       88273   NFLX   buy   11 2023-09-08 15:59:56
37651       28035   GOOG  sell   26 2023-09-08 15:59:57

[37652 rows x 5 columns]


## Trades Analysis

Creates the method `day_trades()` which calculates the number of day trades for each account over a given time window. The method accepts a `start_time` and an `end_time` as parameters for the date window, and it will default to using the `trades_df` DataFrame created above if not specified.

In [20]:
import pandas as pd
import math


def day_trades(start_time, end_time, df=trades_df):
    # set index on timestamp
    df = df.set_index('timestamp')
    # get trades within the start and end time window
    data_in_range = df.loc[start_time:end_time]

    output = []
    # find day trades grouped by account id
    for acct in data_in_range['account_id']:
        # grab the trades for that particular account
        trades_per_acct = data_in_range.loc[data_in_range['account_id'] == acct]
        # group by symbol, see if there is a sell side trade and a buy side trade for that symbol
        sell_and_buy = trades_per_acct.groupby(['symbol']).filter(
            lambda x: 'sell' in x['side'].values and 'buy' in x['side'].values)

        # get the day trade count by looking at record count div by 2
        # this is taking the floor of that result which will handle scenarios for example:
        # where there are 2 buy side orders and one sell side order, which would result in only 1 day trade
        # there's probably a more elegant way to do this...
        day_trade_count = math.floor(sell_and_buy['symbol'].count() / 2)
        # append to output account id and day trade count
        output.append(
            {
                'account_id': acct,
                'day_trades': day_trade_count
            }
        )

    # format the name of the output file to include the date range
    filename = f"day_trades_{start_time}_to_{end_time}.csv"
    
    # create a DataFrame from the output and write it to a csv file 
    result = pd.DataFrame(output).groupby(['account_id']).sum().sort_values(by=['account_id'])

    result.to_csv(filename, header=True, index=False)

## Output

Call the day_trades() method with a `start_time` and `end_time` to generate a `.csv` file with the count of day trades in that time window for each account.

In [21]:
day_trades('2023-09-08 14:59:57', '2023-09-08 15:59:57')